In [12]:
from pyspark.sql import SparkSession
allocated_memory = 18 * 0.75

# create a SparkSession
spark = SparkSession.builder.appName("ReadJSON")\
.config("spark.executor.memory", "6g") \
.master("local[*]")  \
.config("spark.driver.memory", "4g") \
.config("spark.network.timeout", "800s")\
.config("spark.executor.heartbeatInterval", "120s")\
.config("spark.executor.extraJavaOptions", "-XX:+UseG1GC")\
.config("spark.driver.extraJavaOptions", "-XX:+UseG1GC")\
.config("spark.memory.fraction", "0.8") \
.getOrCreate()

print('Default Parallelism :', spark.sparkContext.defaultParallelism)

# Check the default partition size
partition_size = spark.conf.get("spark.sql.files.maxPartitionBytes").replace("b","")
print(f"Partition Size: {partition_size} in bytes and {int(partition_size) / 1024 / 1024} in MB")

from pyspark.sql.types import StructType, StructField, StringType, FloatType, BooleanType, IntegerType

# Define the schema
schema = StructType([
    StructField("overall", FloatType(), True),
    StructField("verified", BooleanType(), True),
    StructField("reviewTime", StringType(), True),
    StructField("reviewerID", StringType(), True),
    StructField("asin", StringType(), True),
    StructField("style", StructType([StructField("Color:", StringType(), True)]), True),
    StructField("reviewerName", StringType(), True),
    StructField("reviewText", StringType(), True),
    StructField("unixReviewTime", IntegerType(), True)

])
json_training_file_path ="../combined_train_data_chunked_10mb_latest.json"

Default Parallelism : 8
Partition Size: 134217728 in bytes and 128.0 in MB


In [13]:
import pyspark.sql.functions as F
from pyspark.sql.types import StructType, StructField, StringType, FloatType, BooleanType, IntegerType, ArrayType
# targetUDF = F.udf(lambda x: 1 if x >= 4.0 else (0 if x == 3.0 else -1), IntegerType())
targetUDF = F.udf(lambda x: 1 if x >= 4.0 else 0, IntegerType())
import re
import nltk
from nltk.corpus import stopwords

def preProcess(text):
    # Should return a list of tokens
    text = re.sub(r"(\w)([.,;:!?'\"”\)])", r"\1 \2", text)
    text = re.sub(r"([.,;:!?'\"“\(])(\w)", r"\1 \2", text)
    text = text.lower()
    tokens = word_tokenize(text)
    return tokens

In [14]:
import re

def removeRegex(tokens: list) -> list:
    """
    Removes hashtags, call outs and web addresses from tokens.
    """
    # Use a raw string for regular expressions to avoid escape sequence warnings
    expr = r'(@[A-Za-z0-9_]+)|(#[A-Za-z0-9_]+)|'+\
           r'(https?://[^\s<>"]+|www\.[^\s<>"]+)'
    regex = re.compile(expr)
    cleaned = [t for t in tokens if not regex.search(t) and len(t) > 0]

    return cleaned

removeWEBUDF = F.udf(removeRegex, ArrayType(StringType()))

def normalize(tokens : list) -> list:
    """
    Removes non-english characters and returns lower case versions of words.
    """
    subbed   = [re.sub("[^a-zA-Z]+", "", s).lower() for s in tokens]

    filtered = filter(None, subbed)

    return list(filtered)


normalizeUDF = F.udf(normalize, ArrayType(StringType()))

In [15]:
from pyspark.ml.feature import Tokenizer
from pyspark.sql.functions import rand

json_test_df = spark.read.schema(schema).json("../combined_test_data_chunked_10mb_latest.json")


tokenizer = Tokenizer(inputCol  = "reviewText",
                      outputCol = "token")
reduced_test_df = json_test_df.select("overall", "reviewerID", "asin", "reviewText")
unique_test_df = reduced_test_df.dropDuplicates(["reviewerID", "asin"])
df_test_sentiment = unique_test_df.withColumn("sentiment", targetUDF(unique_test_df["overall"]))
df_test_tokenized = tokenizer.transform(df_test_sentiment.filter(unique_test_df.reviewText.isNotNull()))
df4_test = df_test_tokenized.withColumn("tokens_re", removeWEBUDF(df_test_tokenized["token"]))
df4_test = df4_test.withColumn("tokens_clean", normalizeUDF(df4_test["tokens_re"]))
df5_test = df4_test.drop("token","tokens_re")
df5_test = df5_test.withColumnRenamed("tokens_clean", "tokens")
df6_test = df5_test.where(F.size(F.col("tokens")) > 0)
df_test_for_model = df6_test.select("reviewText","sentiment").withColumnRenamed("sentiment", "label")
shuffled_test_df = df_test_for_model.orderBy(rand())


In [16]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import NGram, HashingTF, IDF, StopWordsRemover, Tokenizer
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator


sw  = StopWordsRemover(inputCol="tokens", outputCol="filtered")
tf2 = HashingTF(inputCol="filtered", outputCol="rawFeatures", numFeatures=1e5)

evaluator = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction")
bigram = NGram(inputCol="tokens", outputCol="bigrams", n=2)
tf5   = HashingTF(inputCol="bigrams", outputCol="rawFeatures", numFeatures=2e5)
# create tokens from reviews
tk = Tokenizer(inputCol= "reviewText", outputCol = "tokens")
# create tf-idf for each of the tokens
idf = IDF(inputCol="rawFeatures", outputCol="features", minDocFreq=2.0)
# create basic logistic regression model
lr = LogisticRegression(maxIter=20)

bigram_pipeline  = Pipeline(stages= [tk, bigram, tf5, idf, lr])

In [19]:

import sys
import json
import os
import shutil

from pyspark.sql.functions import rand
from pyspark.ml.pipeline import PipelineModel

# Define the maximum file size in bytes (10MB)
max_file_size = 10 * 1024 * 1024

json_objects = []

# Read the file line by line until the maximum file size is reached
with open(json_training_file_path, 'r') as file:
    total_size = 0
    for line in file:
        # Calculate the size of the current line
        line_size = sys.getsizeof(line)

        # If adding the current line exceeds the maximum file size, stop reading
        if total_size + line_size > max_file_size:
            print(line_size, total_size)
            # Create a DataFrame from the list of JSON objects
            df = spark.createDataFrame(json_objects)
            reduced_df = df.select("overall", "reviewerID", "asin", "reviewText")
            unique_df = reduced_df.dropDuplicates(["reviewerID", "asin"])
            df_sentiment = unique_df.withColumn("sentiment", targetUDF(unique_df["overall"]))
            tokenizer = Tokenizer(inputCol  = "reviewText",
                      outputCol = "token")
            # Remove the rows with missing values and tokenize
            df_train_tokenized = tokenizer.transform(df_sentiment.filter(unique_df.reviewText.isNotNull()))
            # remove hashtags, call outs and web addresses
            df4_train = df_train_tokenized.withColumn("tokens_re", removeWEBUDF(df_train_tokenized["token"]))
            # remove non english characters
            df4_train = df4_train.withColumn("tokens_clean", normalizeUDF(df4_train["tokens_re"]))
            # rename columns
            df5_train = df4_train.drop("token","tokens_re")
            df5_train = df5_train.withColumnRenamed("tokens_clean", "tokens")
            # remove reviews where the tokens array is empty, i.e. where it was just
            # a hashtag, callout, numbers, web adress etc.
            df6_train = df5_train.where(F.size(F.col("tokens")) > 0)
            df_train_for_model = df6_train.select("reviewText","sentiment").withColumnRenamed("sentiment", "label")
            shuffled_train_df = df_train_for_model.orderBy(rand())
            if os.path.exists('bigram_pipeline_model'):
                loaded_model = PipelineModel.load('bigram_pipeline_model')
                stages_steps = loaded_model.stages
                updated_model = Pipeline(stages = stages_steps).fit(shuffled_train_df)
                shutil.rmtree('bigram_pipeline_model')
            else:
                updated_model = bigram_pipeline.fit(shuffled_train_df)

            PipelineModel.save(updated_model, 'bigram_pipeline_model')
            print('Model saved')
            json_objects = []
            total_size = 0
        # Otherwise, add the line to the list of JSON objects
        json_objects.append(json.loads(line))
        total_size += line_size

277 10485535


24/04/23 22:17:06 WARN TaskSetManager: Stage 985 contains a task of very large size (1091 KiB). The maximum recommended task size is 1000 KiB.
24/04/23 22:17:15 WARN TaskSetManager: Stage 993 contains a task of very large size (1091 KiB). The maximum recommended task size is 1000 KiB.
24/04/23 22:17:22 WARN TaskSetManager: Stage 998 contains a task of very large size (1091 KiB). The maximum recommended task size is 1000 KiB.
24/04/23 22:17:29 WARN DAGScheduler: Broadcasting large task binary with size 3.2 MiB
24/04/23 22:17:31 WARN DAGScheduler: Broadcasting large task binary with size 3.2 MiB
24/04/23 22:17:33 WARN DAGScheduler: Broadcasting large task binary with size 3.2 MiB
24/04/23 22:17:33 WARN DAGScheduler: Broadcasting large task binary with size 3.2 MiB
24/04/23 22:17:33 WARN DAGScheduler: Broadcasting large task binary with size 3.2 MiB
24/04/23 22:17:33 WARN DAGScheduler: Broadcasting large task binary with size 3.2 MiB
24/04/23 22:17:33 WARN DAGScheduler: Broadcasting large

Model saved
1185 10484917


24/04/23 22:17:59 WARN TaskSetManager: Stage 1124 contains a task of very large size (3196 KiB). The maximum recommended task size is 1000 KiB.
24/04/23 22:18:00 WARN TaskSetManager: Stage 1128 contains a task of very large size (1281 KiB). The maximum recommended task size is 1000 KiB.


Model saved
469 10485345


...

Model saved
473 10485313


24/04/23 23:56:17 WARN TaskSetManager: Stage 20388 contains a task of very large size (3196 KiB). The maximum recommended task size is 1000 KiB.
24/04/23 23:56:18 WARN TaskSetManager: Stage 20392 contains a task of very large size (1281 KiB). The maximum recommended task size is 1000 KiB.


Model saved
1850 10485563


24/04/23 23:56:25 WARN TaskSetManager: Stage 20416 contains a task of very large size (3196 KiB). The maximum recommended task size is 1000 KiB.
24/04/23 23:56:26 WARN TaskSetManager: Stage 20420 contains a task of very large size (1281 KiB). The maximum recommended task size is 1000 KiB.


Model saved


In [20]:
loaded_model = PipelineModel.load('bigram_pipeline_model')
predictions = loaded_model.transform(shuffled_test_df)
score = evaluator.evaluate(predictions)
print("AUC SCORE: {}".format(score))

24/04/24 00:09:16 WARN DAGScheduler: Broadcasting large task binary with size 4.4 MiB


AUC SCORE: 0.7342407927637674
